In [1]:
from gensim.corpora import Dictionary
from gensim.models import Word2Vec

import pandas as pd

import pickle

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
cgl = pd.read_json("../data/large_data/cgl.json")

In [27]:
word_freqs_vocabulary = pd.read_csv("../data/word_freqs_vocabulary.csv")

In [28]:
word_freqs_vocabulary

,word,count
0,λέγω,29114
1,ἔχω,22584
2,γίγνομαι,21737
3,πολύς,20652
4,πᾶς,19062
...,...,...
3475,στραγγουρία,50
3476,ῥύγχος,50
3477,διάκρισις,50
3478,ἀρχός,50


In [13]:
word_freq_dict = dict(zip(word_freqs_vocabulary["word"], word_freqs_vocabulary["count"]))

In [6]:
vocabulary = word_freqs_vocabulary["word"].tolist()

In [42]:
ids_lines = pickle.load(open("../data/ids_lines.pickle", "rb"))

In [78]:
class NgramCorpus(object):
    def __init__(self, ids_list, ids_lines, fname, bow=False, dct=None):
        self.ids_list = ids_list
        self.fname = fname
        self.bow  = bow
        self.dct = dct
        self.ids_lines = ids_lines
        self.len = 0
    def __len__(self):
        [el for el in self]
        return self.len
    def __iter__(self):
        with open(self.fname, "r") as f:
            lines = f.readlines()
            for id in self.ids_list:
                for line in lines[self.ids_lines[id][0] : self.ids_lines[id][1]]:
                    if bool(line.split()):
                        self.len += 1
                        if (self.bow) & (self.dct != None):
                            yield self.dct.doc2bow(line.split())
                        else:
                            yield line.split()

# Full model

In [79]:
corpus_full = NgramCorpus(cgl["doc_id"].tolist(), ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt")

In [80]:
len(corpus_full)

3216205

In [68]:
len([doc for doc in corpus_full])

3216205

In [98]:
def from_corpus_to_keyed_vectors(corpus):
    model = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
    model.build_vocab_from_freq(word_freq=word_freq_dict)
    model.train(corpus, total_examples=len(corpus), epochs=model.epochs)
    keyed_vectors = model.wv
    return keyed_vectors

In [99]:
%%time
keyed_vectors = from_corpus_to_keyed_vectors(corpus_full)

CPU times: user 1min 14s, sys: 2.43 s, total: 1min 17s
Wall time: 54.3 s


In [100]:
keyed_vectors.save("../data/keyed_vectors_full.wv")

# Model excluding c_aristotelicum

In [101]:
set(cgl["subcorpus"])

{None, 'c_aristotelicum', 'c_hippocraticum', 'c_platonicum'}

In [102]:
# ids_list = cgl[cgl["subcorpus"] != "c_aristotelicum"]["doc_id"].tolist()
ids_list = cgl[cgl["author_id"] != "tlg0086"]["doc_id"].tolist()
print(len(ids_list))
corpus_excl_arist = NgramCorpus(ids_list, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt")

336


In [103]:
%%time
keyed_vectors = from_corpus_to_keyed_vectors(corpus_excl_arist)
keyed_vectors.save("../data/keyed_vectors_excl_arist.wv")

CPU times: user 59.3 s, sys: 1.95 s, total: 1min 1s
Wall time: 38.5 s


# Model excluding c_hippocraticum

In [104]:
# ids_list = cgl[cgl["subcorpus"] != "c_hippocraticum"]["doc_id"].tolist()
ids_list = cgl[cgl["author_id"] != "tlg0627"]["doc_id"].tolist()
print(len(ids_list))
corpus_excl_hipp = NgramCorpus(ids_list, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt")

319


In [105]:
%%time
keyed_vectors = from_corpus_to_keyed_vectors(corpus_excl_hipp)
keyed_vectors.save("../data/keyed_vectors_excl_hipp.wv")

CPU times: user 1min 6s, sys: 2.04 s, total: 1min 8s
Wall time: 43.5 s


# Model excluding c_platonicum

In [106]:
# ids_list = cgl[cgl["subcorpus"] != "c_platonicum"]["doc_id"].tolist()
ids_list = cgl[cgl["author_id"] != "tlg0059"]["doc_id"].tolist()
print(len(ids_list))
corpus_excl_plato = NgramCorpus(ids_list, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt")

334


In [107]:
%%time
keyed_vectors = from_corpus_to_keyed_vectors(corpus_excl_plato)
keyed_vectors.save("../data/keyed_vectors_excl_plato.wv")

CPU times: user 1min 3s, sys: 1.94 s, total: 1min 5s
Wall time: 40.7 s
